In [ ]:
#基于文本数据，建立LSTM模型预测序列文字
#1. 完成数预处理将文字序列转化成LSTM输入的数据
#2. 进行数据分离
#3. 针对字符串预测后续字符
# 模型结构单层LSTM输出有20个神经元每次使用前20个字符预测第21个

In [1]:
# read the text
data = open('./data/LSTM/flare.txt').read()
data =data.replace('\n','').replace('\r','')
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country in Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a count

In [2]:
# z字符去重处理
letter = list(set(data))
print(letter)
num_letters = len(letter)
print(num_letters)

['o', 'S', 'y', 'r', 'b', 'i', 'e', 'H', 'c', 'l', 'd', 'p', 'A', '.', ' ', 'm', 'h', 'a', 'n', 'u', 'f', 's', 't']
23


In [3]:
#建立字典
# int to char
int_to_char = {a:b for a,b in enumerate(letter)}
print(int_to_char)
char_to_int = {b:a for a,b in enumerate(letter)}
print(char_to_int)

{0: 'o', 1: 'S', 2: 'y', 3: 'r', 4: 'b', 5: 'i', 6: 'e', 7: 'H', 8: 'c', 9: 'l', 10: 'd', 11: 'p', 12: 'A', 13: '.', 14: ' ', 15: 'm', 16: 'h', 17: 'a', 18: 'n', 19: 'u', 20: 'f', 21: 's', 22: 't'}
{'o': 0, 'S': 1, 'y': 2, 'r': 3, 'b': 4, 'i': 5, 'e': 6, 'H': 7, 'c': 8, 'l': 9, 'd': 10, 'p': 11, 'A': 12, '.': 13, ' ': 14, 'm': 15, 'h': 16, 'a': 17, 'n': 18, 'u': 19, 'f': 20, 's': 21, 't': 22}


In [4]:
# time step
time_step = 20

In [5]:
# 批量处理字符
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data, slide):
    x=[]
    y=[]
    for i in range(len(data)-slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y,char_to_int):
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]]) 
    return x_to_int,y_to_int
#实现输入字符文章的批量处理，输入整个字符，滑动窗口大小，转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0],char_Data[1],char_to_int)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(Input),slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output


In [6]:
# extract x and y from text data
x,y = data_preprocessing(data, time_step,num_letters,char_to_int)
print(y)


[np.int64(18), np.int64(14), np.int64(17), np.int64(5), np.int64(14), np.int64(5), np.int64(18), np.int64(10), np.int64(19), np.int64(21), np.int64(22), np.int64(3), np.int64(2), np.int64(13), np.int64(14), np.int64(7), np.int64(6), np.int64(14), np.int64(0), np.int64(4), np.int64(22), np.int64(17), np.int64(5), np.int64(18), np.int64(6), np.int64(10), np.int64(14), np.int64(16), np.int64(5), np.int64(21), np.int64(14), np.int64(11), np.int64(16), np.int64(10), np.int64(14), np.int64(5), np.int64(18), np.int64(14), np.int64(12), np.int64(19), np.int64(21), np.int64(22), np.int64(3), np.int64(17), np.int64(9), np.int64(5), np.int64(17), np.int64(13), np.int64(12), np.int64(19), np.int64(21), np.int64(22), np.int64(3), np.int64(5), np.int64(9), np.int64(5), np.int64(17), np.int64(14), np.int64(5), np.int64(21), np.int64(14), np.int64(17), np.int64(14), np.int64(8), np.int64(0), np.int64(19), np.int64(18), np.int64(22), np.int64(3), np.int64(2), np.int64(14), np.int64(5), np.int64(18), np

In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y_encode = encoder.fit_transform(np.array(y).reshape(-1,1))
print(y_encode)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4426 stored elements and shape (4426, 23)>
  Coords	Values
  (0, 18)	1.0
  (1, 14)	1.0
  (2, 17)	1.0
  (3, 5)	1.0
  (4, 14)	1.0
  (5, 5)	1.0
  (6, 18)	1.0
  (7, 10)	1.0
  (8, 19)	1.0
  (9, 21)	1.0
  (10, 22)	1.0
  (11, 3)	1.0
  (12, 2)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 7)	1.0
  (16, 6)	1.0
  (17, 14)	1.0
  (18, 0)	1.0
  (19, 4)	1.0
  (20, 22)	1.0
  (21, 17)	1.0
  (22, 5)	1.0
  (23, 18)	1.0
  (24, 6)	1.0
  :	:
  (4401, 2)	1.0
  (4402, 14)	1.0
  (4403, 5)	1.0
  (4404, 18)	1.0
  (4405, 14)	1.0
  (4406, 1)	1.0
  (4407, 0)	1.0
  (4408, 19)	1.0
  (4409, 22)	1.0
  (4410, 16)	1.0
  (4411, 6)	1.0
  (4412, 3)	1.0
  (4413, 18)	1.0
  (4414, 14)	1.0
  (4415, 7)	1.0
  (4416, 6)	1.0
  (4417, 15)	1.0
  (4418, 5)	1.0
  (4419, 21)	1.0
  (4420, 11)	1.0
  (4421, 16)	1.0
  (4422, 6)	1.0
  (4423, 3)	1.0
  (4424, 6)	1.0
  (4425, 13)	1.0


In [8]:
# split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=10)


In [9]:
y_train_category = to_categorical(y_train, num_letters)
print(y_train_category)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
# set up the model
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(units=20,input_shape=(x_train.shape[1],x_train.shape[2]), activation='relu'))
model.add(Dense(units=num_letters,activation='softmax'))
model.compile(optimizer='adam',loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()
          

/opt/anaconda3/envs/AI/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 23)             │           483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,003 (15.64 KB)

 Trainable params: 4,003 (15.64 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(x_train,y_train_category,batch_size=100, epochs=50)

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0823 - loss: 3.1052
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1296 - loss: 2.9227
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1557 - loss: 2.7717
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1564 - loss: 2.6718
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2352 - loss: 2.5379
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3573 - loss: 2.3054
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4743 - loss: 1.9350
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6171 - loss: 1.4203
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6882 - loss: 1.0981
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7442 - loss: 0.8392
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7936 - loss: 0.6783
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8433 - lo

In [12]:
# make prediction based on the test data
y_train_predict = model.predict(x_train)
y_train_predict_norm = (y_train_predict>0.5).astype(int)
y_train_predict_norm = np.argmax(y_train_predict_norm,axis = 1)
print(y_train_predict_norm)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step
[18  7  0 ... 17  6  5]


In [13]:
# transform int to char
y_train_predict_char = [int_to_char[i] for i in y_train_predict_norm]
print(y_train_predict_char)


['n', 'H', 'o', 's', ' ', 't', '.', 'a', 's', 'e', 'n', 'i', ' ', 'o', 'i', 'r', 't', 't', 'd', 't', 'H', 'c', 'u', ' ', ' ', ' ', ' ', ' ', 'o', ' ', ' ', ' ', 's', 's', ' ', 't', 'e', 'i', ' ', 'i', 'p', 'i', 'r', 't', ' ', 'i', 'o', 's', 'h', 's', 'l', 'r', 'c', ' ', 'n', 'i', 's', 's', 'a', 'o', 'u', 'e', 'i', 'i', 'i', 'o', 'i', 'e', 'e', 'A', 'u', 'n', 's', 'n', ' ', 'm', 'n', 'i', 'o', 'i', 'i', 'r', '.', 'n', 't', 'a', 'p', 'n', ' ', 's', 'r', 's', ' ', 's', 'a', 'h', 'i', 'i', 'o', ' ', 'i', 'n', 'p', 'h', 'd', 'r', 'n', 'n', ' ', 'i', 'i', ' ', 't', 't', ' ', 'c', ' ', ' ', 'n', 'n', 'y', 'i', 's', 'a', 'o', 'a', 'r', 'l', 'u', ' ', 't', 's', 'a', ' ', ' ', 'H', 't', 'c', 's', 'i', 'i', 'n', 'u', ' ', ' ', 'u', ' ', ' ', 'e', 'p', 'h', 'r', '.', 'a', 't', 's', 'h', 'a', 'i', 'm', 'i', ' ', '.', 'i', ' ', 'a', 'u', 'o', 'i', 's', 'r', 'u', 'l', 'c', 'l', ' ', 's', 'i', ' ', 'd', ' ', 'a', ' ', ' ', 'h', 'a', ' ', 'a', ' ', 't', 'n', 'i', 't', 'r', 'h', '.', 'u', ' ', 'n', ' ',

In [14]:
# accuracy
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train,y_train_predict_norm)
print(accuracy_train)

1.0


In [15]:
# predict the test set
y_test_predict = model.predict(x_test)
y_test_predict_norm = (y_test_predict>0.5).astype(int)
y_test_predict_norm = np.argmax(y_test_predict_norm,axis = 1)

accuracy_test = accuracy_score(y_test,y_test_predict_norm)
print(accuracy_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1.0


In [16]:
new_letters= 'flare is a teacher in ai industry. He obtained his phd in Australia.Austrilia is a country'
x_new,y_new = data_preprocessing(new_letters,time_step,num_letters,char_to_int)
y_new_predict = model.predict(x_new)
y_new_predict_norm = (y_new_predict>0.5).astype(int)
y_new_predict_norm = np.argmax(y_new_predict_norm,axis = 1)
print(y_new_predict_norm)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
[18 14 17  5 14  5 18 10 19 21 22  3  2 13 14  7  6 14  0  4 22 17  5 18
  6 10 14 16  5 21 14 11 16 10 14  5 18 14 12 19 21 22  3 17  9  5 17 13
 12 19 21 22  3  5  9  5 17 14  5 21 14 17 14  8  0 19 18 22  3  2]


In [17]:
y_new_predict_char = [int_to_char[i] for i in y_new_predict_norm]
print(y_new_predict_char)

['n', ' ', 'a', 'i', ' ', 'i', 'n', 'd', 'u', 's', 't', 'r', 'y', '.', ' ', 'H', 'e', ' ', 'o', 'b', 't', 'a', 'i', 'n', 'e', 'd', ' ', 'h', 'i', 's', ' ', 'p', 'h', 'd', ' ', 'i', 'n', ' ', 'A', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a', '.', 'A', 'u', 's', 't', 'r', 'i', 'l', 'i', 'a', ' ', 'i', 's', ' ', 'a', ' ', 'c', 'o', 'u', 'n', 't', 'r', 'y']


In [18]:
for i in range(0, x_new.shape[0]-20):
    print(new_letters[i:i+20], '--next letter predicted is --- ',y_new_predict_char[i])

flare is a teacher i --next letter predicted is ---  n
lare is a teacher in --next letter predicted is ---   
are is a teacher in  --next letter predicted is ---  a
re is a teacher in a --next letter predicted is ---  i
e is a teacher in ai --next letter predicted is ---   
 is a teacher in ai  --next letter predicted is ---  i
is a teacher in ai i --next letter predicted is ---  n
s a teacher in ai in --next letter predicted is ---  d
 a teacher in ai ind --next letter predicted is ---  u
a teacher in ai indu --next letter predicted is ---  s
 teacher in ai indus --next letter predicted is ---  t
teacher in ai indust --next letter predicted is ---  r
eacher in ai industr --next letter predicted is ---  y
acher in ai industry --next letter predicted is ---  .
cher in ai industry. --next letter predicted is ---   
her in ai industry.  --next letter predicted is ---  H
er in ai industry. H --next letter predicted is ---  e
r in ai industry. He --next letter predicted is ---   
 in ai ind